In [51]:


import keras

In [52]:
pip install Keras


Note: you may need to restart the kernel to use updated packages.


In [53]:
from keras.models import Sequential


In [54]:
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape

In [55]:
from keras.preprocessing.image import ImageDataGenerator

In [56]:
from keras.callbacks import LearningRateScheduler

In [57]:
model = Sequential()
model.add(Conv2D(32, kernel_size=3, padding='same', activation='relu', input_shape=(32, 32, 1)))
model.add(MaxPool2D(pool_size=2))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(43, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [58]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** (x + epochs))
epochs = 15

In [59]:
import pickle

In [60]:
with open('ToySet1.pickle', 'rb') as f:
    data = pickle.load(f, encoding='latin1')

In [61]:

data['y_train'] = to_categorical(data['y_train'], num_classes=43)
data['y_validation'] = to_categorical(data['y_validation'], num_classes=43)

# Making channels come at the end
data['x_train'] = data['x_train'].transpose(0, 2, 3, 1)
data['x_validation'] = data['x_validation'].transpose(0, 2, 3, 1)
data['x_test'] = data['x_test'].transpose(0, 2, 3, 1)

# Showing loaded data from file
for i, j in data.items():
    if i == 'labels':
        print(i + ':', len(j))
    else: 
        print(i + ':', j.shape)

x_train: (540, 32, 32, 1)
y_train: (540, 43)
x_validation: (90, 32, 32, 1)
y_validation: (90, 43)
x_test: (180, 32, 32, 1)
y_test: (180,)
labels: 43


In [62]:
h = model.fit(data['x_train'][:10], data['y_train'][:10],
              batch_size=5, epochs = epochs,
              validation_data = (data['x_validation'], data['y_validation']),
              callbacks=[annealer], verbose=1)

Train on 10 samples, validate on 90 samples
Epoch 1/15
10/10 [==============================] - 0s 46ms/step - loss: 48.5968 - accuracy: 0.1000 - val_loss: 94.0589 - val_accuracy: 0.3333
Epoch 2/15
10/10 [==============================] - 0s 32ms/step - loss: 59.1913 - accuracy: 0.5000 - val_loss: 20.2273 - val_accuracy: 0.3667
Epoch 3/15
10/10 [==============================] - 0s 31ms/step - loss: 6.5431 - accuracy: 0.6000 - val_loss: 63.8403 - val_accuracy: 0.3333
Epoch 4/15
10/10 [==============================] - 0s 29ms/step - loss: 24.7200 - accuracy: 0.4000 - val_loss: 12.3600 - val_accuracy: 0.6333
Epoch 5/15
10/10 [==============================] - 0s 24ms/step - loss: 1.0040 - accuracy: 0.9000 - val_loss: 51.2566 - val_accuracy: 0.6556
Epoch 6/15
10/10 [==============================] - 0s 24ms/step - loss: 43.9677 - accuracy: 0.7000 - val_loss: 64.9111 - val_accuracy: 0.6556
Epoch 7/15
10/10 [==============================] - 0s 24ms/step - loss: 41.1630 - accuracy: 0.7000 